In [2]:
from ultralytics import YOLO
import torch
import cv2 as cv
import cvzone
from sort import *

In [ ]:
# Set device to GPU if available, otherwise use CPU
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

# Load the YOLO model with the specified weights
model = YOLO("../yoloweights/yolov8l.pt").to(device)

# List of class names for object detection
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", 
              "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"]

# Load the mask and graphic images
mask = cv.imread('highway_mask.png')
graphic_image = cv.imread('graphics.png', cv.IMREAD_UNCHANGED)

# Initialize the SORT tracker with specified parameters
tracker = Sort(max_age=20, min_hits=2, iou_threshold=0.3)

# Start video capture from the specified video file
cap = cv.VideoCapture('video_highway.MOV')

# Get video properties
width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv.CAP_PROP_FPS))

# Create a VideoWriter object to save the output video
output_video = cv.VideoWriter("highway_detected_video.mp4", cv.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

# Define the upper and lower limit lines for counting objects
limits_UP = [680, 250, 1130, 250]
limits_D = [160, 250, 620, 250]

# Lists to keep track of total counts for objects moving up and down
total_countUp = []
total_countDown = []

# Main loop to process video frames
while True:
    success, img = cap.read()  # Read a frame from the video
    if not success:  # Break the loop if no frame is captured
        break
    
    # Overlay the graphic image onto the current frame
    img = cvzone.overlayPNG(img, graphic_image, (10, 10))
    
    # Create a masked region of the image
    imageRegion = cv.bitwise_and(img, mask)

    # Initialize an empty array for detections
    detections = np.empty((0, 5))
    
    # Perform object detection on the masked image region
    results = model(imageRegion, stream=True, conf=0.2)

    # Process the detection results
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x, y, w, h = box.xywh[0]  # Get bounding box coordinates
            x, y, w, h = int(x), int(y), int(w), int(h)

            x1, y1, x2, y2 = box.xyxy[0]  # Get corner coordinates
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            conf = round(float(box.conf), 2)  # Confidence score
            cls = int(box.cls[0])  # Class index
            currentClass = classNames[cls]  # Get class name
            
            # Check if the detected object is a vehicle and confidence is above threshold
            if (currentClass in ['car', 'truck', 'bus', 'motorbike']) and (conf > 0.3):
                # Draw bounding box and label on the image
                cvzone.putTextRect(img, f'{currentClass} {conf}', (max(0, int(x - w/2)), max(35, int(y - h/2))), scale=0.7, thickness=1, offset=3)
                
                # Store detection information in the array
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))

    # Draw limit lines on the image
    cv.line(img, (limits_UP[0], limits_UP[1]), (limits_UP[2], limits_UP[3]), (0, 0, 255), thickness=3)
    cv.line(img, (limits_D[0], limits_D[1]), (limits_D[2], limits_D[3]), (0, 0, 255), thickness=3)

    # Update tracker with current detections
    resultsTracker = tracker.update(detections)

    # Process the tracked results
    for result in resultsTracker:
        x1, y1, x2, y2, ID = result  # Get bounding box and ID
        x1, y1, x2, y2, ID = int(x1), int(y1), int(x2), int(y2), int(ID)
        w, h = x2 - x1, y2 - y1  # Calculate width and height

        # Draw rectangle and ID on the image
        cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=2, colorR=(255, 0, 0))
        cvzone.putTextRect(img, f'{ID}', (max(0, x1), max(35, y1)), scale=1, thickness=3, offset=10)
        
        # Calculate the center point of the bounding box
        cx, cy = x1 + w//2, y1 + h//2
        cv.circle(img, (cx, cy), 5, (255, 0, 0), -1)  # Draw circle at center point

        # Check if the object crosses the upper limit line
        if limits_UP[0] < cx < limits_UP[2] and limits_UP[1] - 15 < cy < limits_UP[1] + 15:
            if total_countUp.count(ID) == 0:  # If not counted before
                total_countUp.append(ID)  # Add ID to the count list
                cv.line(img, (limits_UP[0], limits_UP[1]), (limits_UP[2], limits_UP[3]), (0, 255, 0), thickness=3)

        # Check if the object crosses the lower limit line
        if limits_D[0] < cx < limits_D[2] and limits_D[1] - 15 < cy < limits_D[1] + 15:
            if total_countDown.count(ID) == 0:  # If not counted before
                total_countDown.append(ID)  # Add ID to the count list
                cv.line(img, (limits_D[0], limits_D[1]), (limits_D[2], limits_D[3]), (0, 255, 0), thickness=3)

    # Display the counts of vehicles moving up and down on the image
    cv.putText(img, str(len(total_countUp)), (200, 100), cv.FONT_HERSHEY_PLAIN, 5, (139, 195, 75), 7)
    cv.putText(img, str(len(total_countDown)), (450, 100), cv.FONT_HERSHEY_PLAIN, 5, (50, 50, 230), 7)

    # Show the processed image in a window
    cv.imshow("Image", img)

    # Write the processed frame to the output video
    output_video.write(img)

    # Exit the loop if 'q' key is pressed
    if cv.waitKey(1) & 0xff == ord('q'):
        break

# Release resources and close windows
cv.destroyAllWindows()
cap.release()
output_video.release()

In [ ]:
model.device